# Analisi dei corrispettivi dell'azienda <<Là di Cjastelan>>

## Intro

### Descrizione dei dati grezzi

I dati vanno da gennaio 2018 a maggio 2025.

Dal 2018 a maggio 2022 i dati sono stati raccolti manualmente da Riccardo Castellani, da giugno 2022 a marzo 2024 la raccolta, sempre manuale, è stata fatta da Dario Castellani, mentre da aprile 2024 a oggi viene utilizzato un Google Form per far inserire direttamente agli operatori di cassa i vari corrispettivi della giornata.

La raccolta manuale è stata fatta su diversi fogli elettronici Excel. I file prodotti da Riccardo (2018-2022) raccolgono i corrispettivi di un intero anno solare e sono composti da 12 fogli elettronici, uno per ciascun mese dell'anno. I file prodotti da Dario sono annuali per il 2022 e il 2023, divisi per trimestri per il 2024; manca il file relativo all'ultimo trimestre del 2024, essendo questo stato compilato direttamente su cloud in un GoogleDoc. L'ultimo file Excel prodotto da Dario riguarda il primo trimestre del 2025.

Il file del 2022 esiste in doppia versione, una prodotta da Riccardo e una da Dario. La versione di Riccardo verrà ignorata nell'analisi: si tratta, infatti, di una collezione parziale, mancando dell'ultimo trimestre e avendo registrazioni non complete nel trimestre estivo; il file di Dario, invece, duplica tutti i dati corretti della versione di Riccardo e completa e corregge le informazioni mancanti, oltre ad avere i dati dell'ultimo trimestre.

La struttura interna dei fogli di calcolo è simile per ciascun foglio, al netto di aggiunte di casse e reparti che sono arrivate negli anni. I dati sono organizzati in forma tabulare, divisi sulle righe per giorno e sulle colonne per reparti di IVA, raggruppati per cassa. Ai margini ha i totali complessivi e per reparto.

Nel 2018 e nel 2019, la cassa 1 ha 3 reparti: `IVA 4`, `IVA 10` e `IVA 22`, in `IVA 10` collassano i corrispettivi dei reparti 2 e 3. La cassa 2 ha, invece, 4 reparti: `IVA 4`, due `IVA 10` con lo stesso nome ma diversi, probabilmente il primo associato  al Reparto 3 e il secondo al Reparto 2, e un `IVA 22`.

Nel 2020 anche la Cassa 1 presenta una doppia colonna `IVA 10`, probabilmente la prima associata al Reparto 2 e la seconda al Reparto 3. La Cassa 2, invece, sembra invertire l'ordine di associazione dei reparti, avendo nella prima colonna `IVA 10` i corrispettivi del Reparto 2 e, nella seconda colonna `IVA 10`, i corrispettivi del Reparto 3. Dal mese di luglio compare una colonna `gelato` che corrisponde al Reparto 4. Nonostante la presenza del Reparto 4 (che ricade nell'ivaggio commerciale), nella sezione dei totali non compare la colonna `IVA 10 Comm` e i corrispettivi del Reparto 4 vengono fatti confluire nel computo dell'`IVA 10`.

Nel 2021 le colonne con l'IVA al 10% confondono continuamente il Reparto 2 col Reparto 3, da marzo a giugno tutto collassa in un solo reparto. A luglio la colonna `gelato` diventa `DESSERT` ma afferisce sempre al Reparto 4; da agosto torna a chiamarsi `gelato`.

Nel 2022 la colonna `gelato` viene rinominata `COMM` fino ad aprile, quando torna a essere `gelato`. Da settembre si aggiunge ai dati la Cassa 3, che ha inizialmente sola due colonne `IVA 4` e `IVA 10`; poiché questa cassa viene usata solo in mercati dove si vende carne, la colonna corrisponde al Reparto 2. Da novembre la colonna effettivamente prende il nome di `Reparto 2` e si aggiunge anche la colonna `Reparto 5`. A dicembre la Cassa 3 non ha più la colonna `IVA 4`.

Dal 2023 le colonne cominciano a essere intestate ai reparti e non più all'ivaggio corrispondente.

**MANCA DESCRIZIONE DEI DATI WEB**

---

## Importazione dei dati

### Dati Excel

Cominciamo selezionando solo i file che ci interessano, tra tutti quelli consegnati.
Di seguito la struttura della directory `data/raw` dove ho salvato, ed estratto, il file `corrispettivi.zip` contenente tutti i dati consegnati dall'azienda.

In [1]:
import os

DATA_DIR = os.path.join(os.getcwd(), 'data')
RAW_DIR = os.path.join(DATA_DIR, 'raw')

Per motivi di privacy, i dati non sono presenti nel repository. Per questo motivo, riportiamo una rappresentazione testuale della struttura della directory `data/raw`, in modo da poter seguire più agevolmente il processo di selezione dei file utili.

In [2]:
!tree -a {RAW_DIR}

/media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Fond-Scienze-dei-Dati-progetto/data/raw
├── Corrispettivi_web.xlsx
├── corrispettivi.zip
├── Dario
│   ├── CORRISPETTIVI_2022.xlsx
│   ├── CORRISPETTIVI_2023.xlsx
│   ├── corrispettivi 2024.xlsx
│   ├── Corrispettivi apr-mag-giu 2024.xlsx
│   ├── corrispettivi gennaio marzo 2025.xlsx
│   └── corrispettivi lug-ago-set 2024.xlsx
├── .~lock.Corrispettivi_web.xlsx#
└── Riccardo
    ├── 2019
    │   ├── corrispettivi_ago_19.pdf
    │   ├── corrispettivi_apr_19.pdf
    │   ├── corrispettivi_dic_19.pdf
    │   ├── corrispettivi_feb_19.pdf
    │   ├── corrispettivi_gen_19.pdf
    │   ├── corrispettivi_giu_19.pdf
    │   ├── corrispettivi_lug_19.pdf
    │   ├── corrispettivi_mag_19.pdf
    │   ├── corrispettivi_mar_19.pdf
    │   ├── corrispettivi_nov_19.pdf
    │   ├── corrispettivi_ott_19.pdf
    │   └── corrispettivi_set_19.pdf
    ├── 2020
    │   ├── corrispettivi_ago_2020.pdf
    │   ├── corrispettivi_apr_20.pdf
    │  

I file sono divisi in due principali subdirectory nominate in base a chi ha compilato le tabelle.

Nella subdirectory `Riccardo` sono presenti anche file `.pdf` che altro non sono che le esportazioni dei singoli fogli elettronici nei file Excel, inviati poi al commercialista. I file `288_2.pdf` e `griglia_acq.pdf` offrono una sintesi di altri dati commerciali e di fatturazione, che però non riguardano i corrispettivi ma altri atti di compravendita: probabilmente sono stati inseriti per errore. Il file `CORRISPETTIVI.xlsx` invece è un modello vuoto usato come base per la compilazione di tutti gli altri file

In sintesi, dunque, dobbiamo selezionare solo i file Excel (tutti terminanti in `.xlsx`), avendo cura di scartare i file `Riccardo/CORRISPETTIVI_2022.xlsx` e `Riccardo/CORRISPETTIVI.xlsx` che non sono informativi e i file `.~lock` di sistema.

_Nota a posteriori_: i dati grezzi raccolti sul web sono stati molto ripuliti e ho notato che molti errori erano stati tramandati sui fogli Excel già preparati a partire dai dati web. Per questo motivo, ignoriamo tutti i file Excel che vanno oltre marzo 2024 e ricostruiamo tutti i dati partendo dalla fonte primaria.

In [3]:
with os.scandir(RAW_DIR) as rd:
    data_files = []

    for fname in rd:
        if fname.is_dir():
            if fname.name == 'Dario':
                with os.scandir(fname) as dario_sub_dir:
                    data_files += [
                        f.path for f in dario_sub_dir
                            if not f.name.startswith('.')
                            and not '2025' in f.name
                            and not '-' in f.name        # escludo i due file dei trimestri 2024
                    ]
            else: # siamo in `/Riccardo`
                with os.scandir(fname) as riccardo_sub_dir:
                    data_files += [
                        f.path for f in riccardo_sub_dir
                            if f.is_file()                  # escludo le altre subdirectory
                            and '.xlsx' in f.name           # escludo i .pdf
                            and '_' in f.name               # escludo il modello vuoto
                            and not f.name.startswith('.')  # escludo i file di sistema
                            and not '2022' in f.name        # escludo il file parziale
                    ] 

for f in data_files:
    print(f)

len(data_files)

/media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Fond-Scienze-dei-Dati-progetto/data/raw/Dario/corrispettivi 2024.xlsx
/media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Fond-Scienze-dei-Dati-progetto/data/raw/Dario/CORRISPETTIVI_2022.xlsx
/media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Fond-Scienze-dei-Dati-progetto/data/raw/Dario/CORRISPETTIVI_2023.xlsx
/media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Fond-Scienze-dei-Dati-progetto/data/raw/Riccardo/CORRISPETTIVI_2018.xlsx
/media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Fond-Scienze-dei-Dati-progetto/data/raw/Riccardo/CORRISPETTIVI_2019.xlsx
/media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Fond-Scienze-dei-Dati-progetto/data/raw/Riccardo/CORRISPETTIVI_2020.xlsx
/media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Fond-Scienze-dei-Dati-progetto/data/raw/Riccardo/CORRISPETTIVI_2021.xlsx


7

Dei più di 50 file consegnati, abbiamo ora selezionato e salvato in `data_files` i 7 file utili che dovremo importare in Pandas.

Per requisito, la forma del DataFrame finale deve rispecchiare quella del file più recente (del 2025). Usiamo allora un'indicizzazione gerarchica sulle colonne, indicizzando per cassa (dalla 1 alla 3) e specificando, per ogni cassa, gli incassi di ciascun reparto dall'1 al 4. Ci sarà infine un gruppo marginale `Totali`, diviso a sua volta in `IVA 4%`, `IVA 10%`, `IVA 10% comm.` e il totale giornaliero.

La funzione `import_data`, a cui è delegata l'importazione dei file Excel, è molto complessa: ciò dipende dal fatto che i file, e i fogli all'interno di essi, non hanno tutti esattamente la stessa struttura, ma presentano lievi irregolarità sufficienti a rendere impossibile scrivere una procedura abbastanza generale da essere chiamata su ogni file. Per questo motivo, di seguito ci si soffermerà sugli aspetti più salienti della procedura, delegando al codice e ai commenti al codice gli aspetti più puntuali.

Svolgiamo l'importazione usando il metodo `pd.read_excel()`. In questo modo possiamo sfruttare i dati direttamente nel formato con cui ci sono stati passati, senza doverli manipolare manualmente per convertirli in formati più facili da importare, come per esempio in `.csv`. Questo metodo, inoltre, ci fornisce una serie di funzionalità già pronte per poter gestire e tradurre direttamente le particolarità di come sono stati organizzati i dati nei file Excel, come per esempio la divisione in più fogli elettronici in ciascun file.

Chiamiamo `pd.read_excel()` con queste opzioni:
 - `sheet_name=None` per importare tutti i fogli elettronici di cui ciascun file è composto. Questa opzione fa in modo che il metodo non 
 ritorni un DataFrame ma un dizionario, le cui chiavi sono i nomi dei fogli di cui è composto il file e i valori sono il DataFrame importato;
 - `header=[2,3]` per usare le righe 3 e 4 del file Excel per costruire un `MultiIndex` per le colonne del DataFrame;
 - `skipfooter=1` per non importare l'ultima riga di ogni foglio che contiene i totali del mese.

_Nota_: per poter usare il metodo `pd.read_excel()` potrebbe essere necessario installare, tramite `pip`, la dipendenza opzionale `openpyxl`.

Mentre importiamo i dati, dobbiamo codificare le informazioni sul mese e sull'anno che non sono presenti nei dati, ma rispettivamente nel nome dei fogli elettronici e nel nome del file.

In [4]:
import numpy as np
import pandas as pd
from datetime import datetime
import locale

locale.setlocale(locale.LC_TIME, 'it_IT.utf8')
    
def flatten(l):
    '''
    Appiattisce una lista di liste
    '''
    return [el for subl in l for el in subl]    
    
def make_multicols():
    '''
    Crea modularmente un MultiIndex della forma:
    Data                  x Giorno, Mese, Anno, 
    Cassa 1, ..., Cassa 3 x Reparto 1, ..., Reparto 5 (in base a quante n_casse chiediamo)
    Totali                x IVA 4, IVA 10, IVA 10 comm, Tot Giornaliero


    n_casse : int
        il numero di casse registrate
    '''

    level1 = ['Data'] + [f'Cassa {n}' for n in range(1, 4)] + ['Totali']
    level2 = ['Giorno', 'Mese', 'Anno', 'Reparto 1', 'Reparto 2', 'Reparto 3', 'Reparto 4', 'Reparto 5', 'IVA 4', 'IVA 10', 'IVA 10 comm', 'Tot Giornaliero']

    level1_casse_codes = flatten([[n] * 5 for n in range(1, 4)])
    level2_casse_codes = [3, 4, 5, 6, 7] * 3
    level1_codes = [0] * 3 + level1_casse_codes + [4] * 4
    level2_codes = [0, 1, 2] + level2_casse_codes + [8, 9, 10, 11]

    return pd.MultiIndex(levels=[level1, level2], codes=[level1_codes, level2_codes])  

def import_excel_datas(excel_files):  

    def import_date(sheet, df, fname, imported):
        '''
        Compila le colonne Data di tidy in base ai dati sporchi del df che si sta importando

        sheet: chiave del dictionary di df ritornato da pd.read_excel()
        df: valore associato a sheet
        fname: nome del file che si sta importando
        '''

        imported[('Data', 'Giorno')] = df.iloc[:, 0]
        imported[('Data', 'Mese')] = sheet.lower()
        imported[('Data', 'Anno')] = int(fname.split('.')[-2][-4:])

    def import_casse(df, imported):
        '''
        Compila le colonne dei corrispettivi delle casse in tidy associando i dati del df che si sta importando da pd.read_excel()
        '''
 
        iva_rep_map = {
            'IVA 4': 'Reparto 1',        
            'IVA 10_C': 'Reparto 2',     
            'IVA 10_Y': 'Reparto 3',     
            'IVA 10 COMM': 'Reparto 4',
            'IVA 22': 'Reparto 5',
            'gelato': 'Reparto 4',
            'DESSERT': 'Reparto 4',
            'COMM': 'Reparto 4'
        }

        anno = imported[('Data', 'Anno')][0]
        mese = imported[('Data', 'Mese')][0]
        casse = [tup for tup in df.columns if 'Cassa' in tup[0].capitalize()]
        
        for cassa in casse:
            if anno == 2023 and mese == 'marzo' and cassa == ('CASSA 3', 'Rearto 5'):
                # typo nella colonna che va trovato subito
                # print(f"{mese} {cassa} -> {(cassa[0].capitalize(), 'Reparto 5')}")
                imported[(cassa[0].capitalize(), 'Reparto 5')] = df[cassa]
            
            elif not 'reparto' in cassa[1].lower():
                # se i corrispettivi non sono segnati sotto un reparto
                to_map = cassa

                if anno <= 2019:
                    # Per il 2018 e per il 2019, per la Cassa 1 mettiamo tutto in Reparto 2, successivamente in fase di analisi penseremo
                    # come gestire i dati
                    if cassa == ('CASSA 1', 'IVA 10'):
                        to_map = ('CASSA 1', 'IVA 10_C')
               
                    # Per la Cassa 2 nel 2018 e 2019 mappiamo le colonne: prima IVA 10 -> yogurt, seconda IVA 10 -> carne
                    if cassa == ('CASSA 2', 'IVA 10'):
                        to_map = ('CASSA 2', 'IVA 10_Y')
                    if cassa == ('CASSA 2', 'IVA 10 '):
                        to_map = ('CASSA 2', 'IVA 10_C')
                        
                elif anno == 2020:
                    # Cassa 1
                    # In cassa 1 compare la seconda colonna IVA 10, a volte in maiuscolo a volte in minuscolo
                    if cassa == ('CASSA 1', 'IVA 10'):
                        if mese in ['gennaio', 'febbraio', 'marzo']:
                            to_map = ('CASSA 1', 'IVA 10_C')
                        else: # da aprile in poi la prima colonna IVA è scritta in minuscolo e si sfalsa tutto:
                            to_map = ('CASSA 1', 'IVA 10_Y')
                    # se la prima colonna iva 10 è in maiuscolo, la seconda è importata come (IVA 10.1) e va in yogurt
                    elif cassa == ('CASSA 1', 'IVA 10.1'):
                        to_map = ('CASSA 1', 'IVA 10_Y')
                    elif cassa == ('CASSA 1', 'iva 10'):
                        to_map = ('CASSA 1', 'IVA 10_C')
                        
                        
                    # Cassa 2
                    elif cassa == ('CASSA 2', 'IVA 10'):
                        to_map = ('CASSA 2', 'IVA 10_C')
                    elif cassa == ('CASSA 2', 'IVA 10 '):
                        to_map = ('CASSA 2', 'IVA 10_Y')

                elif anno == 2021:
                    # Cassa 1
                    if cassa == ('CASSA 1', 'iva 10'):
                        to_map = ('CASSA 1', 'IVA 10_C')
                    elif cassa == ('CASSA 1', 'IVA 10'):
                        # manca da marzo a giugno
                        to_map = ('CASSA 1', 'IVA 10_Y')

                    # Cassa 2
                    elif cassa == ('CASSA 2', 'IVA 10'):
                        if mese in ['febbraio', 'marzo']: # colonne invertite
                            to_map = ('CASSA 2', 'IVA 10_Y')
                        else:
                            # da aprile va tutto in carne
                            to_map = ('CASSA 2', 'IVA 10_C')
                    elif cassa == ('CASSA 2', 'IVA 10 '):
                        if mese in ['febbraio', 'marzo']: # colonne invertite
                            to_map = ('CASSA 2', 'IVA 10_C')
                        else:
                            to_map = ('CASSA 2', 'IVA 10_Y')
                    
                elif anno == 2022:
                    # Cassa 1
                    if cassa == ('CASSA 1', 'iva 10'):
                        to_map = ('CASSA 1', 'IVA 10_C')
                    elif cassa == ('CASSA 1', 'IVA 10'):
                        to_map = ('CASSA 1', 'IVA 10_Y')

                    # Cassa 2
                    elif cassa == ('CASSA 2', 'IVA 10'):
                        to_map = ('CASSA 2', 'IVA 10_C')
                    elif cassa == ('CASSA 2', 'IVA 10 '):
                        to_map = ('CASSA 2', 'IVA 10_Y')

                    # Cassa 3
                    elif cassa == ('CASSA 3', 'IVA 10'): 
                        to_map = ('CASSA 3', 'IVA 10_C')
                        
                # print(f"{mese} {cassa}->{to_map[0].capitalize(), iva_rep_map[to_map[1]]}")
                imported[(to_map[0].capitalize(), iva_rep_map[to_map[1]])] = df[cassa]
                
            else: # Se sono segnati già sotto il reparto corretto
                # print(f"{mese} {cassa}->{cassa[0].capitalize(), cassa[1]} [else]")
                imported[(cassa[0].capitalize(), cassa[1])] = df[cassa]

    def import_tot(df, imported):
        '''
        Compila le colonne dei totali dei corrispettivi in tidy associando i dati del df che si sta importando da pd.read_excel()
        '''
        tot_map = {
            'CORRISPETTIVI': 'Totali',
            'TOTALE': 'Tot Giornaliero',
            'IVA 10 commerciale': 'IVA 10 comm'
        }

        # Controllo se la prima multicolonna si chiama 'CORRISPETTIVI' invece che 'Totali'
        if 'CORRISPETTIVI' in df.columns.get_level_values(0):
            tots = [tup for tup in df.columns if 'CORRISPETTIVI' in tup[0]]
            for tot in tots:
                mapped = (tot_map[tot[0]], tot[1] if tot[1] not in tot_map.keys() else tot_map[tot[1]])
                imported[mapped] = df[tot]
        else:
            tots = [tup for tup in df.columns if 'Totali' in tup[0].capitalize()]
            for tot in tots:
                imported[(tot[0].capitalize(), tot[1])] = df[tot]
            
            
    '''
    Main Routine
    '''
    
    # creo il DataFrame da restituire come risultato
    res = pd.DataFrame(columns=make_multicols())
    
    for f in excel_files:
        print(f'leggo {f}')
        
        df_dict = None
        if 'apr' in f or 'lug' in f or '2025' in f:
            # i due file a trimestri non hanno le due righe di titolo
            df_dict = pd.read_excel(f, sheet_name=None, header=[0, 1], skipfooter=1)
        else:
            df_dict = pd.read_excel(f, sheet_name=None, header=[2, 3], skipfooter=1)
        
        if '2020' in f:
            # Nel file del 2020, in alcuni fogli, le righe marginali da saltare sono 2
            for key, df in df_dict.items():
                if key in ['febbraio', 'marzo']:
                    df.drop(31, inplace=True)
        elif '2021' in f:
            # Nel file del 2021 c'è un foglio aggiuntivo finale con tutti i totali dell'anno, non ci interessa
            del df_dict['Foglio1']

        elif '2024' in f and 'Foglio4' in df_dict.keys():
            # Nel file 'corrispettivi 2024' c'è un Foglio4 vuoto
            del df_dict['Foglio4']
        
        for sheet, df in df_dict.items():
            df.fillna(0, inplace=True)

            if 'apr' in f:
                # il file del secondo trimestre del 2024 ha la formattazione dell'intestazione delle colonne sballata
                df = pd.DataFrame(df.values, columns=make_multicols().drop([('Data', 'Mese'), ('Data', 'Anno')]))
                
            # creo un DataFrame ben formattato dove copiare i dati importati da poter poi concatenare al risultato
            imported = pd.DataFrame(columns=make_multicols())
            
            import_date(sheet, df, f, imported)
            import_casse(df, imported)
            import_tot(df, imported)

            res = pd.concat([res, imported], ignore_index=True)

    # Sistemiamo i tipi delle date
    res['Data', 'Giorno'] = res['Data', 'Giorno'].astype('int32')
    res['Data', 'Mese'] = res['Data', 'Mese'].astype(pd.StringDtype())
    res['Data', 'Anno'] = res['Data', 'Anno'].astype('int32')

    # riordiniamo le ultime due colonne scambiate
    sorted_cols = list(res.columns)
    sorted_cols[-2], sorted_cols[-1] = sorted_cols[-1], sorted_cols[-2]
    res = res[sorted_cols]
    return res

imported_excels_df = import_excel_datas(data_files)

display(imported_excels_df)
print(imported_excels_df.dtypes)

leggo /media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Fond-Scienze-dei-Dati-progetto/data/raw/Dario/corrispettivi 2024.xlsx
leggo /media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Fond-Scienze-dei-Dati-progetto/data/raw/Dario/CORRISPETTIVI_2022.xlsx


/tmp/ipykernel_12489/412338662.py:225: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res = pd.concat([res, imported], ignore_index=True)
/tmp/ipykernel_12489/412338662.py:225: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res = pd.concat([res, imported], ignore_index=True)
/tmp/ipykernel_12489/412338662.py:225: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determinin

leggo /media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Fond-Scienze-dei-Dati-progetto/data/raw/Dario/CORRISPETTIVI_2023.xlsx
leggo /media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Fond-Scienze-dei-Dati-progetto/data/raw/Riccardo/CORRISPETTIVI_2018.xlsx


/tmp/ipykernel_12489/412338662.py:225: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res = pd.concat([res, imported], ignore_index=True)
/tmp/ipykernel_12489/412338662.py:225: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res = pd.concat([res, imported], ignore_index=True)
/tmp/ipykernel_12489/412338662.py:225: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determinin

leggo /media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Fond-Scienze-dei-Dati-progetto/data/raw/Riccardo/CORRISPETTIVI_2019.xlsx


...

/tmp/ipykernel_12489/412338662.py:225: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res = pd.concat([res, imported], ignore_index=True)
/tmp/ipykernel_12489/412338662.py:225: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res = pd.concat([res, imported], ignore_index=True)
/tmp/ipykernel_12489/412338662.py:225: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determinin

leggo /media/yuuki/Data/Università_vera/II_Anno/Fondamenti_Scienza_Dei_Dati/Fond-Scienze-dei-Dati-progetto/data/raw/Riccardo/CORRISPETTIVI_2021.xlsx


/tmp/ipykernel_12489/412338662.py:225: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res = pd.concat([res, imported], ignore_index=True)
/tmp/ipykernel_12489/412338662.py:225: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  res = pd.concat([res, imported], ignore_index=True)
/tmp/ipykernel_12489/412338662.py:225: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determinin

Data                   Cassa 1                                          \
     Giorno      Mese  Anno Reparto 1 Reparto 2 Reparto 3 Reparto 4 Reparto 5   
0         1   gennaio  2024      0.00      0.00       0.0      0.00       0.0   
1         2   gennaio  2024      0.00      0.00       0.0      0.00       0.0   
2         3   gennaio  2024      0.00      0.00       0.0      0.00       0.0   
3         4   gennaio  2024    137.60     32.30       0.0      0.00     100.0   
4         5   gennaio  2024      0.00      0.00       0.0      0.00       0.0   
...     ...       ...   ...       ...       ...       ...       ...       ...   
2320     27  dicembre  2021    112.76     16.90       2.0     82.90       0.0   
2321     28  dicembre  2021     67.06      0.00       8.0     28.40       0.0   
2322     29  dicembre  2021    350.37     13.72       6.0     15.20       0.0   
2323     30  dicembre  2021    185.86     29.23      16.5     11.00       0.0   
2324     31  dicembre  2021    183.13     28.90       9.8     45.03       0.0   

       Cassa 2            ...   Cassa 3                                \
     Reparto 1 Reparto 2  ... Reparto 1 Reparto 2 Reparto 3 Reparto 4   
0         0.00      0.00  ...       NaN       0.0       NaN       NaN   
1         0.00      0.00  ...       NaN       0.0       NaN       NaN   
2         0.00      0.00  ...       NaN       0.0       NaN       NaN   
3       246.13    112.92  ...       NaN       0.0       NaN       NaN   
4         0.00      0.00  ...       NaN       0.0       NaN       NaN   
...        ...       ...  ...       ...       ...       ...       ...   
2320      0.00      0.00  ...       NaN       NaN       NaN       NaN   
2321    135.60    102.15  ...       NaN       NaN       NaN       NaN   
2322     94.20     12.80  ...       NaN       NaN       NaN       NaN   
2323      0.00      0.00  ...       NaN       NaN       NaN       NaN   
2324      0.00      0.00  ...       NaN       NaN       NaN       NaN   

                Totali                                             
     Reparto 5   IVA 4  IVA 10 IVA 10 comm IVA 22 Tot Giornaliero  
0          0.0    0.00    0.00        0.00    NaN            0.00  
1          0.0    0.00    0.00        0.00    NaN            0.00  
2          0.0    0.00    0.00        0.00    NaN            0.00  
3          0.0  483.73  145.22      159.08    NaN          788.03  
4          0.0    0.00    0.00        0.00    NaN            0.00  
...        ...     ...     ...         ...    ...             ...  
2320       NaN  112.76  101.80         NaN    0.0          214.56  
2321       NaN  202.66  138.55         NaN    0.0          341.21  
2322       NaN  444.57   47.72         NaN    0.0          492.29  
2323       NaN  185.86   56.73         NaN    0.0          242.59  
2324       NaN  183.13   83.73         NaN    0.0          266.86  

[2325 rows x 23 columns]

Data     Giorno                      int32
         Mese               string[python]
         Anno                        int32
Cassa 1  Reparto 1                 float64
         Reparto 2                 float64
         Reparto 3                 float64
         Reparto 4                 float64
         Reparto 5                 float64
Cassa 2  Reparto 1                 float64
         Reparto 2                 float64
         Reparto 3                 float64
         Reparto 4                 float64
         Reparto 5                 float64
Cassa 3  Reparto 1                 float64
         Reparto 2                 float64
         Reparto 3                  object
         Reparto 4                  object
         Reparto 5                 float64
Totali   IVA 4                     float64
         IVA 10                    float64
         IVA 10 comm               float64
         IVA 22                    float64
         Tot Giornaliero           float64
dtype: obje

Per comodità salviamo il DataFrame con tutti i dati importati in un pickle

In [5]:
if 'imported_excel.pickle' not in os.listdir(DATA_DIR):
    imported_excels_df.to_pickle(os.path.join(DATA_DIR, 'imported_excel.pickle'))

### Dati dal web

I dati registrati nei fogli Excel non sono completi: mancano i corrispettivi dell'ultimo trimestre del 2024 e aprile e maggio del 2025. Raccogliamo questi dati dal foglio risposte del Google Form:

In [6]:
web_f = os.path.join(RAW_DIR, 'Corrispettivi_web.xlsx')
web_df = pd.read_excel(web_f, parse_dates=False)

with pd.option_context('display.max_columns', None, 'display.min_rows', 20): 
    display(web_df)
print(web_df.dtypes)

,Informazioni cronologiche,A che giorno corrisponde questo corrispettivo?,Che cassa hai usato?,A quale mercato corrisponde questo corrispettivo?,Totale Giornaliero,Reparto 1,Reparto 2,Reparto 3,Reparto 4,Reparto 5,Totale
0,2024-04-04 09:13:00.292,2024-04-02 00:00:00,Cassa 2,Centro - Martedì,173.43,116.9,46.00,3.3,7.23,NaN,173.43
1,2024-04-04 10:57:09.637,2024-04-02 00:00:00,Cassa 1,Passons - Martedì,227.38,150.97,14.07,20.6,41.74,NaN,227.38
2,2024-04-04 11:03:20.802,2024-04-03 00:00:00,Cassa 1,Villaggio - Mercoledì,174.58,108.81,15.94,17.4,32.43,NaN,174.58
3,2024-04-04 12:07:19.551,2024-04-04 00:00:00,Cassa 1,S. Osvaldo - Giovedì,285.38,170.63,47.62,18.9,48.23,NaN,285.38
4,2024-04-04 13:20:35.190,2024-04-04 00:00:00,Cassa 2,Centro - Giovedì,134.25,115.47,NaN,7.0,11.78,NaN,134.25
5,2024-04-06 12:10:20.301,2024-04-06 00:00:00,Cassa 1,Cormor - Sabato,425.68,226.78,75.64,13.2,110.06,NaN,425.68
6,2024-04-06 13:53:06.163,2024-04-06 00:00:00,Cassa 2,Centro - Sabato,516.17,342.48,120.99,19.1,33.6,NaN,516.17
7,2024-04-08 12:37:22.130,4/8/0024,Cassa 2,Centro - Lunedì,124.3,84.93,10.20,8.8,20.33,NaN,124.26
8,2024-04-09 17:08:03.532,4/9/0024,Cassa 2,Passons - Martedì,371.91,294.25,22.06,48.8,6.8,NaN,371.91
9,2024-04-10 14:29:20.024,4/10/0024,Cassa 2,Villaggio - Mercoledì,446.6,267.47,97.73,17.0,64.4,NaN,446.60


Informazioni cronologiche                            datetime64[ns]
A che giorno corrisponde questo corrispettivo?               object
Che cassa hai usato?                                         object
A quale mercato corrisponde questo corrispettivo?            object
Totale Giornaliero                                           object
Reparto 1                                                    object
Reparto 2                                                   float64
Reparto 3                                                   float64
Reparto 4                                                    object
Reparto 5                                                   float64
Totale                                                      float64
dtype: object


L'ultima colonna del foglio, di nome `Totale`, è una colonna di controllo.

Ci sono dei problemi:
- tipo di `Reparto 1` e `Reparto 4` non numerico
- date scritte in modo sbagliato
- date incoerenti:
    - scontrini "del futuro", cioè la data inserita manualmente è successiva al momento dell'inserimento
    - scontrini battuti in giorni in cui non c'è il mercato a cui si riferiscono, nel senso che un corrispettivo di un mercato di un lunedì viene datato a un mercoledì, per esempio (ma potrebbe essere perché è stato spostato il mercato)
- totale disallineato con la colonna di controllo

Prima di importare, cerchiamo di pulire un po' i dati.

In [7]:
# Buttiamo via le righe vuote
web_df['Totale'] = web_df['Totale'].map(lambda x: np.nan if x == 0 else x)
web_df = web_df.dropna(how='all')
display(web_df)

,Informazioni cronologiche,A che giorno corrisponde questo corrispettivo?,Che cassa hai usato?,A quale mercato corrisponde questo corrispettivo?,Totale Giornaliero,Reparto 1,Reparto 2,Reparto 3,Reparto 4,Reparto 5,Totale
0,2024-04-04 09:13:00.292,2024-04-02 00:00:00,Cassa 2,Centro - Martedì,173.43,116.9,46.00,3.30,7.23,NaN,173.43
1,2024-04-04 10:57:09.637,2024-04-02 00:00:00,Cassa 1,Passons - Martedì,227.38,150.97,14.07,20.60,41.74,NaN,227.38
2,2024-04-04 11:03:20.802,2024-04-03 00:00:00,Cassa 1,Villaggio - Mercoledì,174.58,108.81,15.94,17.40,32.43,NaN,174.58
3,2024-04-04 12:07:19.551,2024-04-04 00:00:00,Cassa 1,S. Osvaldo - Giovedì,285.38,170.63,47.62,18.90,48.23,NaN,285.38
4,2024-04-04 13:20:35.190,2024-04-04 00:00:00,Cassa 2,Centro - Giovedì,134.25,115.47,NaN,7.00,11.78,NaN,134.25
...,...,...,...,...,...,...,...,...,...,...,...
456,2025-06-05 10:01:37.879,5/12/0025,Cassa 1,Centro - Lunedì,76.38,58.38,15.00,NaN,3,NaN,NaN
457,2025-06-05 12:43:25.206,2025-06-05 00:00:00,Cassa 2,Centro - Giovedì,180.53,121.19,37.64,7.90,13.8,NaN,NaN
458,2025-06-07 12:47:43.467,2025-06-07 00:00:00,Cassa 2,Centro - Sabato,394.75,302.22,30.88,37.25,24.4,NaN,NaN
459,2025-06-10 12:15:20.788,5/10/0025,Cassa 2,Passons - Martedì,235.89,132.22,31.57,32.40,39.7,NaN,NaN


In [8]:
# controlliamo i reparti 1 e 4
print('Errori nel Reparto 1:')
display(web_df[ ~web_df['Reparto 1'].apply(np.isreal) ])

print('Errori nel Reparto 4:')
display(web_df[ ~web_df['Reparto 4'].apply(np.isreal) ])

Errori nel Reparto 1:


,Informazioni cronologiche,A che giorno corrisponde questo corrispettivo?,Che cassa hai usato?,A quale mercato corrisponde questo corrispettivo?,Totale Giornaliero,Reparto 1,Reparto 2,Reparto 3,Reparto 4,Reparto 5,Totale
157,2024-08-28 08:36:02.805,8/24/0024,Cassa 1,Cormor - Sabato,317.27,"205 ,07",48.15,12.1,6.95,NaN,67.2


Errori nel Reparto 4:


,Informazioni cronologiche,A che giorno corrisponde questo corrispettivo?,Che cassa hai usato?,A quale mercato corrisponde questo corrispettivo?,Totale Giornaliero,Reparto 1,Reparto 2,Reparto 3,Reparto 4,Reparto 5,Totale
158,2024-08-28 08:44:42.890,8/26/0024,Cassa 1,Centro - Lunedì,154.19,99.29,21.15,7.8,25 60,NaN,128.24


In [9]:
# Correggiamo i dati

web_df.loc[157, 'Reparto 1'] = 250.07 # è 250.07 e non 205.07, lo si vede sottraendo gli altri reparti dal totale giornaliero
web_df.loc[158, 'Reparto 4'] = 25.6
web_df = web_df.astype({'Reparto 1': 'float64', 'Reparto 4': 'float64'})

display(web_df.loc[[157, 158], :])
web_df.dtypes

,Informazioni cronologiche,A che giorno corrisponde questo corrispettivo?,Che cassa hai usato?,A quale mercato corrisponde questo corrispettivo?,Totale Giornaliero,Reparto 1,Reparto 2,Reparto 3,Reparto 4,Reparto 5,Totale
157,2024-08-28 08:36:02.805,8/24/0024,Cassa 1,Cormor - Sabato,317.27,250.07,48.15,12.1,6.95,NaN,67.20
158,2024-08-28 08:44:42.890,8/26/0024,Cassa 1,Centro - Lunedì,154.19,99.29,21.15,7.8,25.60,NaN,128.24


Informazioni cronologiche                            datetime64[ns]
A che giorno corrisponde questo corrispettivo?               object
Che cassa hai usato?                                         object
A quale mercato corrisponde questo corrispettivo?            object
Totale Giornaliero                                           object
Reparto 1                                                   float64
Reparto 2                                                   float64
Reparto 3                                                   float64
Reparto 4                                                   float64
Reparto 5                                                   float64
Totale                                                      float64
dtype: object

In [10]:
# Sistemiamo le date nella seconda colonna
def formatta_data(strd):
    if type(strd) is str:
        # la data è nella forma m/d/00yy
        splitted = strd.split('/')
        m = splitted[0]
        d = splitted[1]
        y = f"20{splitted[2][-2:]}"
        return f"{y}-{m}-{d}"
    else:
        return datetime.strftime(strd, '%Y-%m-%d')

web_df['Data'] = web_df.iloc[:,1].map(formatta_data)
web_df['Data'] = pd.to_datetime(web_df['Data'], format='%Y-%m-%d')

with pd.option_context('display.max_columns', None, 'display.min_rows', 20): 
    display(web_df)
print(web_df.dtypes)

,Informazioni cronologiche,A che giorno corrisponde questo corrispettivo?,Che cassa hai usato?,A quale mercato corrisponde questo corrispettivo?,Totale Giornaliero,Reparto 1,Reparto 2,Reparto 3,Reparto 4,Reparto 5,Totale,Data
0,2024-04-04 09:13:00.292,2024-04-02 00:00:00,Cassa 2,Centro - Martedì,173.43,116.90,46.00,3.30,7.23,NaN,173.43,2024-04-02
1,2024-04-04 10:57:09.637,2024-04-02 00:00:00,Cassa 1,Passons - Martedì,227.38,150.97,14.07,20.60,41.74,NaN,227.38,2024-04-02
2,2024-04-04 11:03:20.802,2024-04-03 00:00:00,Cassa 1,Villaggio - Mercoledì,174.58,108.81,15.94,17.40,32.43,NaN,174.58,2024-04-03
3,2024-04-04 12:07:19.551,2024-04-04 00:00:00,Cassa 1,S. Osvaldo - Giovedì,285.38,170.63,47.62,18.90,48.23,NaN,285.38,2024-04-04
4,2024-04-04 13:20:35.190,2024-04-04 00:00:00,Cassa 2,Centro - Giovedì,134.25,115.47,NaN,7.00,11.78,NaN,134.25,2024-04-04
5,2024-04-06 12:10:20.301,2024-04-06 00:00:00,Cassa 1,Cormor - Sabato,425.68,226.78,75.64,13.20,110.06,NaN,425.68,2024-04-06
6,2024-04-06 13:53:06.163,2024-04-06 00:00:00,Cassa 2,Centro - Sabato,516.17,342.48,120.99,19.10,33.60,NaN,516.17,2024-04-06
7,2024-04-08 12:37:22.130,4/8/0024,Cassa 2,Centro - Lunedì,124.3,84.93,10.20,8.80,20.33,NaN,124.26,2024-04-08
8,2024-04-09 17:08:03.532,4/9/0024,Cassa 2,Passons - Martedì,371.91,294.25,22.06,48.80,6.80,NaN,371.91,2024-04-09
9,2024-04-10 14:29:20.024,4/10/0024,Cassa 2,Villaggio - Mercoledì,446.6,267.47,97.73,17.00,64.40,NaN,446.60,2024-04-10


Informazioni cronologiche                            datetime64[ns]
A che giorno corrisponde questo corrispettivo?               object
Che cassa hai usato?                                         object
A quale mercato corrisponde questo corrispettivo?            object
Totale Giornaliero                                           object
Reparto 1                                                   float64
Reparto 2                                                   float64
Reparto 3                                                   float64
Reparto 4                                                   float64
Reparto 5                                                   float64
Totale                                                      float64
Data                                                 datetime64[ns]
dtype: object


In [11]:
# Cerchiamo le entry in cui è stato inserito uno "scontrino del futuro" (data sbagliata)
web_df[ web_df['Data'] > web_df['Informazioni cronologiche']]


,Informazioni cronologiche,A che giorno corrisponde questo corrispettivo?,Che cassa hai usato?,A quale mercato corrisponde questo corrispettivo?,Totale Giornaliero,Reparto 1,Reparto 2,Reparto 3,Reparto 4,Reparto 5,Totale,Data
42,2024-05-11 12:23:00.119,5/11/0025,Cassa 1,Cormor - Sabato,776.09,500.66,165.11,29.4,80.92,NaN,776.09,2025-05-11
71,2024-06-06 12:03:54.391,6/5/0025,Cassa 1,Villaggio - Mercoledì,307.83,255.53,31.90,6.6,13.80,NaN,307.83,2025-06-05
146,2024-08-20 10:39:23.704,9/10/0024,Cassa 1,Coperto - Sabato,256.42,230.62,NaN,6.6,19.20,NaN,256.42,2024-09-10
191,2024-10-03 10:28:40.672,11/2/0024,Cassa 1,Villaggio - Mercoledì,224.92,179.57,18.75,7.3,19.30,NaN,224.92,2024-11-02
205,2024-10-20 18:58:47.321,11/20/0024,Cassa 3,San Pietro,646.67,388.35,205.66,19.4,NaN,33.26,646.67,2024-11-20
325,2025-02-18 10:51:58.658,2/13/0026,Cassa 1,S. Osvaldo - Giovedì,276.35,89.76,150.89,35.7,NaN,NaN,276.35,2026-02-13
340,2025-02-20 09:09:43.884,2/19/0026,Cassa 1,Villaggio - Mercoledì,288.72,168.45,68.17,9.9,42.20,NaN,288.72,2026-02-19
344,2025-02-26 10:46:56.727,2/25/0026,Cassa 2,Passons - Martedì,527.94,376.21,105.13,41.6,5.00,NaN,527.94,2026-02-25


In [12]:
# Correggiamo le entry

web_df.loc[42, 'Data'] = pd.to_datetime('2024-05-11')
web_df.loc[71, 'Data'] = pd.to_datetime('2024-06-05')
web_df.loc[146, 'Data'] = pd.to_datetime('2024-08-10')
web_df.loc[191, 'Data'] = pd.to_datetime('2024-10-02')
web_df.loc[205, 'Data'] = pd.to_datetime('2024-10-20')
web_df.loc[325, 'Data'] = pd.to_datetime('2025-02-13')
web_df.loc[340, 'Data'] = pd.to_datetime('2025-02-19')
web_df.loc[344, 'Data'] = pd.to_datetime('2025-02-25')

web_df.loc[[42, 71, 146, 191, 205, 325, 340, 344], :]

,Informazioni cronologiche,A che giorno corrisponde questo corrispettivo?,Che cassa hai usato?,A quale mercato corrisponde questo corrispettivo?,Totale Giornaliero,Reparto 1,Reparto 2,Reparto 3,Reparto 4,Reparto 5,Totale,Data
42,2024-05-11 12:23:00.119,5/11/0025,Cassa 1,Cormor - Sabato,776.09,500.66,165.11,29.4,80.92,NaN,776.09,2024-05-11
71,2024-06-06 12:03:54.391,6/5/0025,Cassa 1,Villaggio - Mercoledì,307.83,255.53,31.90,6.6,13.80,NaN,307.83,2024-06-05
146,2024-08-20 10:39:23.704,9/10/0024,Cassa 1,Coperto - Sabato,256.42,230.62,NaN,6.6,19.20,NaN,256.42,2024-08-10
191,2024-10-03 10:28:40.672,11/2/0024,Cassa 1,Villaggio - Mercoledì,224.92,179.57,18.75,7.3,19.30,NaN,224.92,2024-10-02
205,2024-10-20 18:58:47.321,11/20/0024,Cassa 3,San Pietro,646.67,388.35,205.66,19.4,NaN,33.26,646.67,2024-10-20
325,2025-02-18 10:51:58.658,2/13/0026,Cassa 1,S. Osvaldo - Giovedì,276.35,89.76,150.89,35.7,NaN,NaN,276.35,2025-02-13
340,2025-02-20 09:09:43.884,2/19/0026,Cassa 1,Villaggio - Mercoledì,288.72,168.45,68.17,9.9,42.20,NaN,288.72,2025-02-19
344,2025-02-26 10:46:56.727,2/25/0026,Cassa 2,Passons - Martedì,527.94,376.21,105.13,41.6,5.00,NaN,527.94,2025-02-25


In [13]:
# bisogna anche controllare che che la data inserita sia coerente con il mercato
# cioè che se lo scontrino si riferisce a un mercato di un martedì, allora la data deve essere di un martedì
# altrimenti potrebbe esserci un errore con la data

web_df['Giorno'] = web_df['Data'].dt.strftime('%A')

giorni_incoerenti_m = web_df.apply(
    lambda x: x['Giorno'].upper() not in x['A quale mercato corrisponde questo corrispettivo?'].upper()
    and 'ì' in x['A quale mercato corrisponde questo corrispettivo?'], # escludiamo i mercati che non hanno il giorno nell'entry della colonna
    axis=1)

web_df[giorni_incoerenti_m ]

,Informazioni cronologiche,A che giorno corrisponde questo corrispettivo?,Che cassa hai usato?,A quale mercato corrisponde questo corrispettivo?,Totale Giornaliero,Reparto 1,Reparto 2,Reparto 3,Reparto 4,Reparto 5,Totale,Data,Giorno
133,2024-08-20 09:51:54.607,7/25/0024,Cassa 1,Villaggio - Mercoledì,186.15,158.95,NaN,27.30,NaN,NaN,186.25,2024-07-25,giovedì
134,2024-08-20 09:53:29.304,7/24/0024,Cassa 1,S. Osvaldo - Giovedì,239.63,228.83,NaN,10.80,NaN,NaN,239.63,2024-07-24,mercoledì
242,2024-12-28 09:11:38.861,12/24/0024,Cassa 1,Villaggio - Mercoledì,318.26,119.24,170.17,13.85,15.00,NaN,318.26,2024-12-24,martedì
278,2024-12-30 09:46:20.577,11/30/0024,Cassa 1,Villaggio - Mercoledì,400.15,299.74,34.10,27.40,38.91,NaN,400.15,2024-11-30,sabato
331,2025-02-18 11:13:54.847,1/29/0025,Cassa 1,S. Osvaldo - Giovedì,299.52,216.93,62.09,15.50,5.00,NaN,299.52,2025-01-29,mercoledì
438,2025-06-05 09:22:10.709,5/26/0025,Cassa 1,Mercoledì Treppo,316.33,192.96,105.57,2.80,15.00,NaN,NaN,2025-05-26,lunedì
459,2025-06-10 12:15:20.788,5/10/0025,Cassa 2,Passons - Martedì,235.89,132.22,31.57,32.40,39.70,NaN,NaN,2025-05-10,sabato


In [14]:
# 133, 134 ->le entry sono chiaramente invertite
web_df.loc[133, 'A quale mercato corrisponde questo corrispettivo?'] = 'S. Osvaldo - Giovedì'
web_df.loc[134, 'A quale mercato corrisponde questo corrispettivo?'] = 'Villaggio - Mercoledì'

# 242-> non è sbagliata, è stato spostato il mercato che altrimenti sarebbe caduto di natale (verificato col cliente)

# 278 -> è sbagliato il mese, si tratta di ottobre (il 30 ottobre 2024 era un mercoledì)
# si capisce guardando le entry vicine: chi stava compilando il form stava inserendo in blocco degli scontrini di ottobre
web_df.loc[278, 'Data'] = pd.to_datetime('2024-10-30')

# 331 -> _dovrebbe_ essere così, sentendo il cliente. Si tratta di un altro inserimento in blocco
web_df.loc[331, 'A quale mercato corrisponde questo corrispettivo?'] = 'Villaggio - Mercoledì'

# 438 -> si tratta del 28 e non del 26 (errore comune). Mercoledì 28 infatti non risulta nessun mercato Treppo
web_df.loc[438, 'Data'] = pd.to_datetime('2025-05-28')

# 459 -> qui semplice refuso sulla data
web_df.loc[459, 'Data'] = pd.to_datetime('2025-06-10')

web_df['Giorno'] = web_df['Data'].dt.strftime('%A')
web_df.loc[[133, 134 ,278, 331, 438, 459], :]

,Informazioni cronologiche,A che giorno corrisponde questo corrispettivo?,Che cassa hai usato?,A quale mercato corrisponde questo corrispettivo?,Totale Giornaliero,Reparto 1,Reparto 2,Reparto 3,Reparto 4,Reparto 5,Totale,Data,Giorno
133,2024-08-20 09:51:54.607,7/25/0024,Cassa 1,S. Osvaldo - Giovedì,186.15,158.95,NaN,27.3,NaN,NaN,186.25,2024-07-25,giovedì
134,2024-08-20 09:53:29.304,7/24/0024,Cassa 1,Villaggio - Mercoledì,239.63,228.83,NaN,10.8,NaN,NaN,239.63,2024-07-24,mercoledì
278,2024-12-30 09:46:20.577,11/30/0024,Cassa 1,Villaggio - Mercoledì,400.15,299.74,34.10,27.4,38.91,NaN,400.15,2024-10-30,mercoledì
331,2025-02-18 11:13:54.847,1/29/0025,Cassa 1,Villaggio - Mercoledì,299.52,216.93,62.09,15.5,5.00,NaN,299.52,2025-01-29,mercoledì
438,2025-06-05 09:22:10.709,5/26/0025,Cassa 1,Mercoledì Treppo,316.33,192.96,105.57,2.8,15.00,NaN,NaN,2025-05-28,mercoledì
459,2025-06-10 12:15:20.788,5/10/0025,Cassa 2,Passons - Martedì,235.89,132.22,31.57,32.4,39.70,NaN,NaN,2025-06-10,martedì


In [15]:
# controlliamo i totali
with pd.option_context('display.max_rows', None):
    tot_m = (web_df['Totale Giornaliero'] != web_df['Totale']) & (web_df['Totale'].notna()) # degli ultimi corrispettivi non è stato calcolato il totale di controllo
    display(web_df[tot_m])
    print(len(web_df[tot_m]))

,Informazioni cronologiche,A che giorno corrisponde questo corrispettivo?,Che cassa hai usato?,A quale mercato corrisponde questo corrispettivo?,Totale Giornaliero,Reparto 1,Reparto 2,Reparto 3,Reparto 4,Reparto 5,Totale,Data,Giorno
7,2024-04-08 12:37:22.130,4/8/0024,Cassa 2,Centro - Lunedì,124.3,84.93,10.200,8.80,20.33,NaN,124.260,2024-04-08,lunedì
11,2024-04-11 16:12:45.600,4/11/0024,Cassa 2,S. Osvaldo - Giovedì,443.42,338.83,28.160,29.20,37.23,NaN,433.420,2024-04-11,giovedì
57,2024-05-25 12:32:46.367,5/25/0024,Cassa 1,Cormor - Sabato,501.89,285.18,80.110,18.18,117.80,NaN,501.270,2024-05-25,sabato
79,2024-06-13 11:45:08.544,6/12/0024,Cassa 1,Villaggio - Mercoledì,286.2,NaN,NaN,15.15,62.70,NaN,77.850,2024-06-12,mercoledì
90,2024-06-22 13:49:53.473,2024-06-22 00:00:00,Cassa 2,Centro - Sabato,322.76,231.35,NaN,22.90,58.51,NaN,312.760,2024-06-22,sabato
91,2024-06-24 10:46:48.837,6/22/0024,Cassa 1,Cormor - Sabato,401.35,328.29,7.150,28.70,37.20,NaN,401.340,2024-06-22,sabato
97,2024-06-29 09:36:49.340,6/27/0024,Cassa 1,S. Osvaldo - Giovedì,214.97,192.97,NaN,22.50,NaN,NaN,215.470,2024-06-27,giovedì
106,2024-07-08 11:18:49.390,7/4/0024,Cassa 1,S. Osvaldo - Giovedì,301.07,204.92,NaN,42.40,53.75,NaN,301.070,2024-07-04,giovedì
111,2024-07-15 08:53:40.624,7/8/0024,Cassa 1,Centro - Lunedì,227.83,6.85,NaN,4.64,6.90,NaN,18.390,2024-07-08,lunedì
113,2024-07-15 08:57:48.345,7/10/0024,Cassa 1,Villaggio - Mercoledì,274.47,238.47,NaN,35.60,NaN,NaN,274.070,2024-07-10,mercoledì


41


Da qui emerge intanto che c'è almeno un'entry in `Reparto 2` che ha 3 cifre decimali invece che due. Cerchiamole e sistemiamo intanto quelle:

In [16]:
r2err = web_df['Reparto 2'].dropna().astype('str').str.split('.', expand=True)[1].apply(lambda x: len(x)) # split con expand mi torna due colonne invece che una lista in una colonna
r2err[ r2err > 2 ]

141    3
Name: 1, dtype: int64

In [17]:
web_df.loc[141]

Informazioni cronologiche                            2024-08-20 10:11:13.036000
A che giorno corrisponde questo corrispettivo?                         8/3/0024
Che cassa hai usato?                                                    Cassa 1
A quale mercato corrisponde questo corrispettivo?               Cormor - Sabato
Totale Giornaliero                                                       259.19
Reparto 1                                                                207.34
Reparto 2                                                                10.251
Reparto 3                                                                  19.6
Reparto 4                                                                   NaN
Reparto 5                                                                  22.0
Totale                                                                  259.191
Data                                                        2024-08-03 00:00:00
Giorno                                  

In [18]:
# correggiamo, ma è impossibile sapere come... tronchiamo alla seconda cifra e pace
web_df.loc[141, 'Reparto 2'] = 10.25
web_df.loc[141, 'Totale'] = 259.19

web_df.loc[141]

Informazioni cronologiche                            2024-08-20 10:11:13.036000
A che giorno corrisponde questo corrispettivo?                         8/3/0024
Che cassa hai usato?                                                    Cassa 1
A quale mercato corrisponde questo corrispettivo?               Cormor - Sabato
Totale Giornaliero                                                       259.19
Reparto 1                                                                207.34
Reparto 2                                                                 10.25
Reparto 3                                                                  19.6
Reparto 4                                                                   NaN
Reparto 5                                                                  22.0
Totale                                                                   259.19
Data                                                        2024-08-03 00:00:00
Giorno                                  

In [19]:
# Ritorniamo ai totali

with pd.option_context('display.max_rows', None):
    tot_m = (web_df['Totale Giornaliero'] != web_df['Totale']) & (web_df['Totale'].notna()) # dobbiamo ricalcolarla
    display(web_df[tot_m])
    print(len(web_df[tot_m]))

,Informazioni cronologiche,A che giorno corrisponde questo corrispettivo?,Che cassa hai usato?,A quale mercato corrisponde questo corrispettivo?,Totale Giornaliero,Reparto 1,Reparto 2,Reparto 3,Reparto 4,Reparto 5,Totale,Data,Giorno
7,2024-04-08 12:37:22.130,4/8/0024,Cassa 2,Centro - Lunedì,124.3,84.93,10.20,8.80,20.33,NaN,124.26,2024-04-08,lunedì
11,2024-04-11 16:12:45.600,4/11/0024,Cassa 2,S. Osvaldo - Giovedì,443.42,338.83,28.16,29.20,37.23,NaN,433.42,2024-04-11,giovedì
57,2024-05-25 12:32:46.367,5/25/0024,Cassa 1,Cormor - Sabato,501.89,285.18,80.11,18.18,117.80,NaN,501.27,2024-05-25,sabato
79,2024-06-13 11:45:08.544,6/12/0024,Cassa 1,Villaggio - Mercoledì,286.2,NaN,NaN,15.15,62.70,NaN,77.85,2024-06-12,mercoledì
90,2024-06-22 13:49:53.473,2024-06-22 00:00:00,Cassa 2,Centro - Sabato,322.76,231.35,NaN,22.90,58.51,NaN,312.76,2024-06-22,sabato
91,2024-06-24 10:46:48.837,6/22/0024,Cassa 1,Cormor - Sabato,401.35,328.29,7.15,28.70,37.20,NaN,401.34,2024-06-22,sabato
97,2024-06-29 09:36:49.340,6/27/0024,Cassa 1,S. Osvaldo - Giovedì,214.97,192.97,NaN,22.50,NaN,NaN,215.47,2024-06-27,giovedì
106,2024-07-08 11:18:49.390,7/4/0024,Cassa 1,S. Osvaldo - Giovedì,301.07,204.92,NaN,42.40,53.75,NaN,301.07,2024-07-04,giovedì
111,2024-07-15 08:53:40.624,7/8/0024,Cassa 1,Centro - Lunedì,227.83,6.85,NaN,4.64,6.90,NaN,18.39,2024-07-08,lunedì
113,2024-07-15 08:57:48.345,7/10/0024,Cassa 1,Villaggio - Mercoledì,274.47,238.47,NaN,35.60,NaN,NaN,274.07,2024-07-10,mercoledì


40


In [20]:
# La maggior parte delle entry si vede che presenta solo errori di battitura, ma per alcune non è così:

# 79 -> Chi ha compilato si è evidentemente dimenticato di inserire il Reparto 1
web_df.loc[79, 'Reparto 1'] = web_df.loc[79, 'Totale Giornaliero'] - web_df.loc[79, 'Totale']

# 111 -> probabilmente è stato inserito il Reparto 2 al posto del Reparto 1 che si è perso
web_df.loc[111, 'Reparto 2'] = web_df.loc[111, 'Reparto 1']
web_df.loc[111, 'Reparto 1'] = web_df.loc[111, 'Totale Giornaliero'] - web_df.loc[111, 'Reparto 2'] - web_df.loc[111, 'Reparto 3'] - web_df.loc[111, 'Reparto 4']

# 145 saltato il Reparto 2
web_df.loc[145, 'Reparto 2'] = web_df.loc[145, 'Totale Giornaliero'] - web_df.loc[145, 'Reparto 1'] - web_df.loc[145, 'Reparto 3'] - web_df.loc[145, 'Reparto 4']

# le 158, 235 e 360 sono impossibili da correggere e vanno tenute così

web_df.loc[[79, 111, 145], :]

,Informazioni cronologiche,A che giorno corrisponde questo corrispettivo?,Che cassa hai usato?,A quale mercato corrisponde questo corrispettivo?,Totale Giornaliero,Reparto 1,Reparto 2,Reparto 3,Reparto 4,Reparto 5,Totale,Data,Giorno
79,2024-06-13 11:45:08.544,6/12/0024,Cassa 1,Villaggio - Mercoledì,286.2,208.35,NaN,15.15,62.70,NaN,77.85,2024-06-12,mercoledì
111,2024-07-15 08:53:40.624,7/8/0024,Cassa 1,Centro - Lunedì,227.83,209.44,6.85,4.64,6.90,NaN,18.39,2024-07-08,lunedì
145,2024-08-20 10:38:13.314,8/8/0024,Cassa 1,S. Osvaldo - Giovedì,404.16,264.33,55.00,38.00,46.83,NaN,349.16,2024-08-08,giovedì


In [21]:
# diamo gli ultimi ritocchi e salviamo il DataFrame ripulito in un formato comodo per le analisi successive
web_df_clean = web_df.copy()

web_df_clean.drop(columns=['Informazioni cronologiche', 'A che giorno corrisponde questo corrispettivo?', 'Totale Giornaliero', 'Totale', 'Giorno'], inplace=True)
web_df_clean.rename(columns={'Che cassa hai usato?': 'Cassa', 'A quale mercato corrisponde questo corrispettivo?': 'Mercato'}, inplace=True)
cols = list(web_df_clean.columns)
cols.insert(0, cols.pop())
web_df_clean = web_df_clean[cols]

web_df_clean

,Data,Cassa,Mercato,Reparto 1,Reparto 2,Reparto 3,Reparto 4,Reparto 5
0,2024-04-02,Cassa 2,Centro - Martedì,116.90,46.00,3.30,7.23,NaN
1,2024-04-02,Cassa 1,Passons - Martedì,150.97,14.07,20.60,41.74,NaN
2,2024-04-03,Cassa 1,Villaggio - Mercoledì,108.81,15.94,17.40,32.43,NaN
3,2024-04-04,Cassa 1,S. Osvaldo - Giovedì,170.63,47.62,18.90,48.23,NaN
4,2024-04-04,Cassa 2,Centro - Giovedì,115.47,NaN,7.00,11.78,NaN
...,...,...,...,...,...,...,...,...
456,2025-05-12,Cassa 1,Centro - Lunedì,58.38,15.00,NaN,3.00,NaN
457,2025-06-05,Cassa 2,Centro - Giovedì,121.19,37.64,7.90,13.80,NaN
458,2025-06-07,Cassa 2,Centro - Sabato,302.22,30.88,37.25,24.40,NaN
459,2025-06-10,Cassa 2,Passons - Martedì,132.22,31.57,32.40,39.70,NaN


In [23]:
if 'web_clean.pickle' not in os.listdir(DATA_DIR):
    web_df_clean.to_pickle(os.path.join(DATA_DIR, 'web_clean.pickle'))

A partire da questo foglio risposte parzialmente ripulito possiamo costruire il DataFrame con i dati mancanti, da concatenare al risultato dell'importazione dei fogli Excel:

In [24]:
def import_from_web(web_df):
    # Creo un nuovo DataFrame della forma di quello dei fogli Excel ma lo indicizzo con le date che mi servono:
    # idx = pd.date_range(start='2024-10-01', end='2024-12-31').union(pd.date_range(start='2025-04-01', end='2025-05-31'))
    idx = pd.date_range(start='2024-04-01', end='2025-05-31')
    
    imported = pd.DataFrame(columns=make_multicols(), index=idx)

    def import_row(row):
        data = row['Data']
        if data in imported.index.values:
            reps = [f'Reparto {i}' for i in range(1, 6)]
            for rep in reps:
                imported.loc[data, (row['Cassa'], rep)] = row[rep]
            
    web_df.apply(import_row, axis=1)

    def fill_date(row):
        data = row.name
        imported.loc[data, ('Data', 'Giorno')] = data.day
        imported.loc[data, ('Data', 'Mese')] = data.strftime('%B') # mese in lettere per esteso
        imported.loc[data, ('Data', 'Anno')] = data.year

    imported.apply(fill_date, axis=1)

    # Sistemiamo i tipi
    
    imported['Data', 'Giorno'] = imported['Data', 'Giorno'].astype('Int32')
    imported['Data', 'Mese'] = imported['Data', 'Mese'].astype(pd.StringDtype())
    imported['Data', 'Anno'] = imported['Data', 'Anno'].astype('Int32')

    imported = imported.apply(lambda x: x.astype('float64') if 'Data' not in x.name else x)
    
    return imported
        
imported_web_df = import_from_web(web_df_clean)

display(imported_web_df)
print(imported_web_df.dtypes)

Data                 Cassa 1                                \
           Giorno    Mese  Anno Reparto 1 Reparto 2 Reparto 3 Reparto 4   
2024-04-01      1  aprile  2024       NaN       NaN       NaN       NaN   
2024-04-02      2  aprile  2024    150.97     14.07      20.6     41.74   
2024-04-03      3  aprile  2024    108.81     15.94      17.4     32.43   
2024-04-04      4  aprile  2024    170.63     47.62      18.9     48.23   
2024-04-05      5  aprile  2024       NaN       NaN       NaN       NaN   
...           ...     ...   ...       ...       ...       ...       ...   
2025-05-27     27  maggio  2025    175.62     24.12      39.7     12.80   
2025-05-28     28  maggio  2025    192.96    105.57       2.8     15.00   
2025-05-29     29  maggio  2025    104.84     57.62      40.0     20.00   
2025-05-30     30  maggio  2025       NaN       NaN       NaN       NaN   
2025-05-31     31  maggio  2025    331.05     35.70       5.1     31.00   

                       Cassa 2            ...             Cassa 3            \
           Reparto 5 Reparto 1 Reparto 2  ... Reparto 5 Reparto 1 Reparto 2   
2024-04-01       NaN       NaN       NaN  ...       NaN       NaN       NaN   
2024-04-02       NaN    116.90     46.00  ...       NaN       NaN       NaN   
2024-04-03       NaN       NaN       NaN  ...       NaN       NaN       NaN   
2024-04-04       NaN    115.47       NaN  ...       NaN       NaN       NaN   
2024-04-05       NaN       NaN       NaN  ...       NaN       NaN       NaN   
...              ...       ...       ...  ...       ...       ...       ...   
2025-05-27       NaN       NaN       NaN  ...       NaN       NaN       NaN   
2025-05-28       NaN    234.64     11.59  ...       NaN       NaN       NaN   
2025-05-29       NaN    145.00     55.84  ...       NaN       NaN       NaN   
2025-05-30       NaN       NaN       NaN  ...       NaN       NaN       NaN   
2025-05-31       NaN    395.83     22.10  ...       NaN       NaN       NaN   

                                         Totali                     \
           Reparto 3 Reparto 4 Reparto 5  IVA 4 IVA 10 IVA 10 comm   
2024-04-01       NaN       NaN       NaN    NaN    NaN         NaN   
2024-04-02       NaN       NaN       NaN    NaN    NaN         NaN   
2024-04-03       NaN       NaN       NaN    NaN    NaN         NaN   
2024-04-04       NaN       NaN       NaN    NaN    NaN         NaN   
2024-04-05       NaN       NaN       NaN    NaN    NaN         NaN   
...              ...       ...       ...    ...    ...         ...   
2025-05-27       NaN       NaN       NaN    NaN    NaN         NaN   
2025-05-28       NaN       NaN       NaN    NaN    NaN         NaN   
2025-05-29       NaN       NaN       NaN    NaN    NaN         NaN   
2025-05-30       NaN       NaN       NaN    NaN    NaN         NaN   
2025-05-31       NaN       NaN       NaN    NaN    NaN         NaN   

                            
           Tot Giornaliero  
2024-04-01             NaN  
2024-04-02             NaN  
2024-04-03             NaN  
2024-04-04             NaN  
2024-04-05             NaN  
...                    ...  
2025-05-27             NaN  
2025-05-28             NaN  
2025-05-29             NaN  
2025-05-30             NaN  
2025-05-31             NaN  

[426 rows x 22 columns]

Data     Giorno                      Int32
         Mese               string[python]
         Anno                        Int32
Cassa 1  Reparto 1                 float64
         Reparto 2                 float64
         Reparto 3                 float64
         Reparto 4                 float64
         Reparto 5                 float64
Cassa 2  Reparto 1                 float64
         Reparto 2                 float64
         Reparto 3                 float64
         Reparto 4                 float64
         Reparto 5                 float64
Cassa 3  Reparto 1                 float64
         Reparto 2                 float64
         Reparto 3                 float64
         Reparto 4                 float64
         Reparto 5                 float64
Totali   IVA 4                     float64
         IVA 10                    float64
         IVA 10 comm               float64
         Tot Giornaliero           float64
dtype: object


Per sicurezza, salviamo il pickle del DataFrame così importato

In [26]:
if 'imported_web.pickle' not in os.listdir(DATA_DIR):
    imported_web_df.to_pickle(os.path.join(DATA_DIR, 'imported_web.pickle'))

Arriviamo dunque al risultato finale:

In [ ]:
imported_df = pd.concat([imported_excels_df, imported_web_df], ignore_index=True)

display(imported_df)
print(imported_df.dtypes)

---
## Pulizia dei Dati

- reindicizzare sulla data (eliminando le date invalide)
- togliere tutti i totali (alcuni non sono calcolati e nel corso degli anni i dati sono stati aggregati in modo disomogeneo)
- applicare le stesse correzioni ai dati fatte su `web_df` che non sono state importate nel dataset finale, perché gli errori sono stati tramandati sui fogli Excel

In [ ]:
corrispettivi_df = imported_df.copy()

def ricostruisci_data(row):
    d = row.iat[0]
    m = row.iat[1]
    y = row.iat[2]
    data = pd.to_datetime(f'{y}-{m}-{d}', format='%Y-%B-%d', errors='coerce') # con coerce i valori invalidi vengono segnati come NaT
    corrispettivi_df.loc[row.name, ('Data', 'Idx')] = data
    
corrispettivi_df.apply(ricostruisci_data, axis=1)
corrispettivi_df.drop(corrispettivi_df[ corrispettivi_df['Data', 'Idx'].isna() ].index,  inplace=True)
corrispettivi_df.sort_values(('Data', 'Idx'), inplace=True)
corrispettivi_df.set_index(('Data', 'Idx'), inplace=True)
corrispettivi_df.index.rename('Data', inplace=True)

corrispettivi_df.drop(columns=[col for col in corrispettivi_df.columns if ('Data' in col[0] or 'Totali' in col[0])], inplace=True)

display(corrispettivi_df)
print(corrispettivi_df.dtypes)

Salviamo i dati puliti per poterli recuperare facilmente e velocemente in seguito

In [ ]:
if 'corrispettivi_clean.pickle' not in os.listdir(DATA_DIR):
    corrispettivi_df.to_pickle(os.path.join(DATA_DIR, 'corrispettivi_clean.pickle'))

---
## Visualizzazione

### Domande
- quali mesi vanno meglio in genere
- quali prodotti vanno meglio in genere
- come va ciascun mercato
- quali prodotti vanno meglio in base al periodo
- quanto il meteo influenza il ricavato

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

### Quali mesi vanno meglio in genere?

In [ ]:
# Faccio resampling del dataset aggregando tutti i giorni in mesi. MS = month start
corr_mensile = corrispettivi_df.resample('MS').sum()
corr_mensile['Totale mensile (€)'] = corr_mensile.sum(axis=1)

print('Dataset ricampionato mensilmente')
corr_mensile

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
sns.lineplot(data=corr_mensile, x='Data', y='Totale mensile (€)')

# Major ticks: Anni
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Minor ticks: Mesi
# ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=3))
ax.xaxis.set_minor_locator(mdates.MonthLocator(bymonth=[3, 6, 8, 10, 12]))
ax.xaxis.set_minor_formatter(mdates.DateFormatter('%b'))  # nomi brevi dei mesi

# Abilita minor ticks
ax.tick_params(axis='x', which='major', labelsize=10, pad=30)
ax.tick_params(axis='x', which='minor', labelsize=8, rotation=45)

# sistemo la griglia
ax.grid(which='minor', axis='x', linestyle='--')
ax.grid(False, axis='y')

# mostro il periodo covid (novembre 2020-aprile 2021)
ax.axvspan(pd.to_datetime('2020-11-1'), pd.to_datetime('2021-4-1'), color='red', alpha=0.2)
ax.text(pd.to_datetime('2021-6-1'), ax.get_ylim()[0] * 1.1, 'Lockdown COVID19', ha='center', va='bottom', fontsize='9')

ax.set_xlim(datetime(2018, 1, 1), datetime(2025, 5, 31))
ax.set_xlabel('')

ax.set_yticklabels([])  

plt.tight_layout()
ax.set_title('Andamento generale delle entrate (per mese)')
plt.show()

In [ ]:
# Per sovrapporre gli anni, ricampioniamo settimanalmente
compar_anni = corrispettivi_df.resample('W').sum()
compar_anni['Totale settimanale (€)'] = compar_anni.sum(axis=1)

# compar_anni.reset_index(inplace=True)
compar_anni['Settimana'] = compar_anni.index.isocalendar().week
compar_anni['Anno'] = compar_anni.index.year
# leviamo il 2025
compar_anni = compar_anni[ compar_anni['Anno'] < 2025 ]

print('Dataset ricampionato settimanalmente')
compar_anni

In [ ]:
plt.figure(figsize=(14, 6))
sns.lineplot(data=compar_anni, x='Settimana', y='Totale settimanale (€)', hue='Anno', palette=sns.color_palette('Set2', 7))
ax = plt.gca()

ax.set_xticks(ticks=range(0, 53, 5))
ax.tick_params(axis='x', which='major', pad=20)
# mesi come minor ticks
ax.set_xticks(np.linspace(1, 49, 12), minor=True)
ax.set_xticklabels([d.strftime('%b') for d in pd.date_range('2024-01-01', '2024-12-31', freq='MS')], minor=True, fontsize=9) # MS month start frequency
ax.set_xlim(1, 53)

ax.set_ylabel(ax.get_ylabel(), labelpad=20)

ax.grid(axis='y', linestyle=(0, (4, 10))) #(0, (3, 10, 1, 15)) means (3pt line, 10pt space, 1pt line, 15pt space) with no offset, while (5, (10, 3)), means (10pt line, 3pt space), but skip the first 5pt line

ax.set_title("Comparazione dell'andamento delle entrate negli anni 2018-2024 (per settimana)")

plt.tight_layout()
plt.show()

Con questi grafici rispondiamo alla domanda: **quali sono i mesi che vanno meglio e peggio?** Generalmente, abbiamo picchi a ottobre, dicembre e marzo (in corrispondenza delle feste, evidentemente) e cali nei mesi estivi, con agosto come mese peggiore (ma anche perché ci sono le ferie).

In [ ]:
compar2425 = corrispettivi_df.copy()

compar2425['Totale giornaliero (€)'] = compar2425.sum(axis=1)
compar2425['Anno'] = compar2425.index.year
compar2425 = compar2425[ (compar2425['Anno'] >= 2024) & (compar2425.index.month <= 5) & compar2425['Totale giornaliero (€)'] > 0]
compar2425['Giorno'] = compar2425.index.day_of_year

print('Dataset per comparare giornalmente 2024 e 2025')
compar2425

In [ ]:
plt.figure(figsize=(14, 6))
sns.lineplot(data=compar2425, x='Giorno', y='Totale giornaliero (€)', hue='Anno')

ax = plt.gca()

compar2425_xticks = [i for i in range(0, 152, 30)]
compar2425_xticks[0] = 4
ax.set_xticks(compar2425_xticks)
ax.set_xticks(range(15, 152, 30), minor=True)
ax.set_xticklabels(['gennaio', 'febbraio', 'marzo', 'aprile', 'maggio'], minor=True, fontsize=10)
ax.set_xlim(4, 152)

ax.set_yticks([500, 1500, 2500])

ax.grid(axis='y', linestyle=(0, (4, 10)))

ax.set_title('Comparazione andamento entrate giornaliere tra il 2024 e il 2025')

plt.tight_layout()
plt.show()

### Quali prodotti vanno meglio in genere?

**Problema**: Per alcuni periodi non abbiamo la separazione tra Reparto 2 e Reparto 3:
- Nel 2018 e nel 2019 la Cassa 1 batte tutto nel Reparto 2
- Nel 2021:
  - Cassa 1 da marzo a giugno batte tutto nel Reparto 2
  - Cassa 2 da aprile in poi batte tutto nel Reparto 2
- Nel 2022 la Cassa 2 torna a separare i reparti da marzo

**Soluzione**: potremmo usare i dati che abbiamo per calcolare come si distribuiscono percentualmente i due reparti negli altri anni e stimare la divisione dove i dati sono collassati, ma così praticamente staremmo manipolando i dati per confermare la nostra ipotesi. Quindi non resta che buttare via tutti i giorni in cui non abbiamo la distinzione tra i reparti

In [ ]:
reparti = corrispettivi_df.copy()

reparti = reparti[
    (reparti.index.year == 2020)
    | ((reparti.index.year == 2021) & (reparti.index.month <= 2))
    | ((reparti.index.year == 2022) & (reparti.index.month >= 3))
    | (reparti.index.year >= 2023)
]

#with pd.option_context('display.max_rows', None): 
#    display(reparti)

reparti_sums = reparti.sum().unstack(level=0)
reparti_sums['Totale'] = reparti_sums.sum(axis=1)
reparti_sums['Prodotti'] = ['Formaggi e latticini', 'Carne e insaccati', 'Yogurt', 'Gelato, dessert\n e gastronomia', 'Altro (non agricolo)']

# conto quanti scontrini sono stati effettivamente battuti per ciascun reparto
reparti_count = reparti.copy()
reparti_count = (reparti_count > 0).sum().unstack(level=0)
reparti_count['Count'] = reparti_count.sum(axis=1)
print('Conteggio di quanti giorni in totale un reparto compare nel corrispettivo')
display(reparti_count)

reparti_sums['Count'] = reparti_count['Count']
reparti_sums['Totale normalizzato'] = reparti_sums['Totale'] / reparti_sums['Count']
entrata_totale = reparti_sums['Totale normalizzato'].sum()
reparti_sums['Entrata normalizzata'] = reparti_sums['Totale normalizzato'] / entrata_totale * 100
scontrini_tot = reparti_sums['Count'].sum()
reparti_sums['Corrispettivi registrati'] = reparti_sums['Count'] / scontrini_tot * 100

print('Dataset con i totali di entrate e conteggi')
display(reparti_sums)

reparti_sums_melted = reparti_sums.melt(id_vars='Prodotti', value_vars=['Entrata normalizzata', 'Corrispettivi registrati'], var_name='Tipo')
print('Stesso dataset massaggiato in forma lunga per poter plottare due serie insieme')
reparti_sums_melted

In [ ]:
# mia palette custom fatta con ColorBrewer, simile ai colori dell'azienda
barplot_palette = ['orange','orangered','brown']

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
abs_ax = axes[0]
n_ax = axes[1]

fig.suptitle('Distribuzione complessiva delle entrate per categoria di prodotto')

# assoluto
sns.barplot(data=reparti_sums, x='Prodotti', y='Totale', color=barplot_palette[0], ax=abs_ax)


abs_ax.set_ylabel('Entrata totale (in decine di migliaia di €)', labelpad=10)
abs_ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda tick, pos: f'{int(tick/10_000)}'))

abs_ax.tick_params(axis='x', labelsize=10)

abs_ax.set_title('Distribuzione assoluta')

# normalizzato
sns.barplot(data=reparti_sums_melted, x='Prodotti', y='value', hue='Tipo', palette={'Entrata normalizzata': barplot_palette[0], 'Corrispettivi registrati': barplot_palette[1]}, ax=n_ax)


n_ax.set_ylabel('')
n_ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda tick, pos: f'{int(tick)}%'))

n_ax.tick_params(axis='x', labelsize=10)

n_ax.set_title('Distribuzione normalizzata relativa affiancata ai pesi')

# tolgo il titolo dalla legenda
handles, labels = n_ax.get_legend_handles_labels()
n_ax.legend(handles=handles, labels=labels)
sns.move_legend(n_ax, "upper left", bbox_to_anchor=(.55, .95))

# finto asse x comune
abs_ax.set_xlabel('')
n_ax.set_xlabel('')
fig.text(0.525, 0, 'Prodotti', ha='center')

fig.tight_layout()
#fig.subplots_adjust(top=0.88)
plt.show()

Il reparto 5 è magagnato. Leviamolo dai dati:

In [ ]:
reparti_sums_no5 = reparti_sums.loc[:'Reparto 4', ['Prodotti', 'Totale normalizzato']]
reparti_sums_no5['Entrata normalizzata (%)'] = reparti_sums_no5['Totale normalizzato'] / reparti_sums_no5['Totale normalizzato'].sum() * 100
print('Stesso dataset ma senza Reparto 5')
reparti_sums_no5

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
sns.barplot(reparti_sums_no5, x='Prodotti', y='Entrata normalizzata (%)', color=barplot_palette[0])

ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda tick, pos: f'{int(tick)}%'))
ax.set_ylabel(ax.yaxis.get_label_text(), labelpad=20)

ax.tick_params(axis='x', labelsize=10)

ax.set_title('Distribuzione normalizzata relativa delle entrate per categorie di prodotti')

fig.tight_layout()
plt.show()

In [ ]:
reparti = corrispettivi_df.copy()

# porto i reparti sulle righe. future_stack sostanzialmente è la nuova implementazione che non chiama dropna e sort
reparti = reparti.stack(future_stack=True).reset_index(1, names=['Data', 'Reparto'])

# teniamo solo gli anni di cui abbiamo dati completi: 2020, 2023 e 2024 e leviamo il reparto 5
reparti['Anno'] = reparti.index.year
reparti = reparti[
    (
        (reparti['Anno'] == 2020)
        | (reparti['Anno'] == 2023)
        | (reparti['Anno'] == 2024)
    ) & (
        reparti['Reparto'] != 'Reparto 5'
    )
]
print('Dataset intermedio: porto i reparti sulle righe')
display(reparti)

rep_per_anno = reparti.pivot_table(index=['Anno', 'Reparto'], aggfunc=['sum', lambda col: (col > 0).sum()])
print('Dataset intermedio ruotato con somma e conteggi')
display(rep_per_anno)

# aggreghiamo le casse, sempre portandole sulle righe
rep_per_anno = rep_per_anno.stack(future_stack=True).groupby(['Anno', 'Reparto']).sum()

rep_per_anno['Entrata normalizzata'] = rep_per_anno['sum'] / rep_per_anno['<lambda>']
entrata_tot_n_per_anno = rep_per_anno.groupby('Anno').agg({'Entrata normalizzata': 'sum'})
rep_per_anno['Entrata normalizzata (%)'] = rep_per_anno['Entrata normalizzata'] / entrata_tot_n_per_anno['Entrata normalizzata'] * 100

rep_per_anno['Prodotti'] = ['Formaggi e latticini', 'Carne e insaccati', 'Yogurt', 'Gelato, dessert\n e gastronomia'] * 3
rep_per_anno.reset_index('Anno', inplace=True)

print('Dataset finale con entrate assolute e normalizzate per reparti raggruppato per anni')
rep_per_anno

In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(rep_per_anno, x='Prodotti', y='Entrata normalizzata (%)', hue='Anno', palette=barplot_palette)
ax = plt.gca()

ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda tick, pos: f'{int(tick)}%'))
ax.set_ylabel('')

ax.tick_params(axis='x', labelsize=10)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels)
sns.move_legend(ax, "upper left", bbox_to_anchor=(.75, .85))

ax.set_title('Andamento per anno della distribuzione normalizzata relativa delle entrate per categorie di prodotti')

plt.tight_layout()
plt.show()

**Risposta**:
- i prodotti che incidono di più sulle entrate sono di gran lunga formaggi e latticini (mozzarella, burro, stracchino...)
- la carne pesa meno di quello che, intuitivamente, saremmo portati a credere, ma comunque ha un grosso peso
- la tendenza negli anni vede pesare meno il formaggio e i latticini e più la carne
- lo yogurt viene venduto molto di più della gastronomia ma porta meno entrate

### Quali mercati vanno meglio?
L'informazione sui mercati ce l'abbiamo solo da **aprile 2024** in poi, cioè da quando si è passati a raccogliere i corrispettivi online dal Google Form. Possiamo quindi ragionare sui 13 mesi di dati.
Il dataset di riferimento cambia, non lavoriamo più con quello completo dei corrispettivi (`corrispettivi_df`) ma prendiamo i dati web importati e puliti di `web_df_clean`.

Individuiamo quali sono i mercati e dividiamo i mercati periodici (che ci interesseano) dai mercati svolti durante eventi isolati (Friuli DOC, fiere, ecc.).

I mercati periodici attualmente tenuti dal cliente sono:
- Mercato coperto (sabato, una volta al mese)
- Centro (lunedì, giovedì e sabato)
- Passons (martedì)
- Villaggio del Sole (mercoledì)
- Treppo Grande (mercoledì)
- San Osvaldo (giovedì)
- Parco del Cormor (sabato)
- Alveari (vendita online con distribuzione sul territorio)

**Problemi**:
- il mercato di Treppo nasce come evento (svolto di domenica) per poi diventare periodico. Dobbiamo quindi distinguere i due mercati
- il mercato del centro si teneva anche di martedì, da qualche tempo hanno rinunciato alla data

In [17]:
#mercati_df = web_df_clean.copy()
mercati_df = pd.read_pickle('data/web_clean.pickle')
mercati = set(mercati_df['Mercato'].values)
for m in mercati:
    print(m)

Centro - Giovedì
Centro - Sabato
SanPietro
Castello
San Pietro
Villaggio
Castello 
Villaggio - Mercoledì
Domenica - Oleis
Centro - Martedì
Treppo Grande
Evento vendoglio
Cormor - Sabato
Alveari
Mercoledì Treppo 
Coperto - Sabato
San Pietri
Treppo mercoledì 
San pieri
Centro - Lunedì
Osoppo
Passons - Martedì
S. Osvaldo - Giovedì
San pietro
Treppo


In [18]:
# Sistemiamo prima la questione Treppo

treppo_df = mercati_df[ mercati_df['Mercato'].str.contains('Treppo') ]
treppo_df['Giorno'] = treppo_df['Data'].dt.strftime('%A')
treppo_df

/tmp/ipykernel_11628/291636263.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treppo_df['Giorno'] = treppo_df['Data'].dt.strftime('%A')


,Data,Cassa,Mercato,Reparto 1,Reparto 2,Reparto 3,Reparto 4,Reparto 5,Giorno
353,2025-03-05,Cassa 2,Treppo Grande,212.99,52.23,17.00,36.9,NaN,mercoledì
361,2025-03-12,Cassa 2,Treppo,127.77,219.53,6.84,NaN,NaN,mercoledì
378,2025-03-26,Cassa 2,Treppo,167.43,49.22,5.10,NaN,NaN,mercoledì
386,2025-04-02,Cassa 1,Treppo mercoledì,223.48,191.16,19.25,17.5,NaN,mercoledì
399,2025-04-09,Cassa 1,Treppo,140.85,48.21,5.75,6.9,NaN,mercoledì
408,2025-04-23,Cassa 1,Treppo,70.89,26.69,NaN,NaN,NaN,mercoledì
413,2025-04-16,Cassa 1,Treppo,162.74,169.71,22.55,NaN,NaN,mercoledì
434,2025-06-04,Cassa 1,Mercoledì Treppo,107.37,41.91,17.10,38.3,NaN,mercoledì
438,2025-05-26,Cassa 1,Mercoledì Treppo,192.96,105.57,2.80,15.0,NaN,lunedì
443,2025-05-21,Cassa 1,Mercoledì Treppo,149.62,232.52,14.15,12.0,NaN,mercoledì
